# Creating and  Using Graph DB in Python Application

## Prepare the Environment

In [9]:
# Try again
#!pip install -U neo4j 
!python -m pip install neo4j

In [10]:
# download the latest Python driver from https://pypi.org/project/neo4j/#files and install it
!pip install "/Users/tdi/Downloads/neo4j-5.18.0.tar"

Processing c:\users\tdi\downloads\neo4j-5.18.0.tar


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\tdi\\Downloads\\neo4j-5.18.0.tar'



In [11]:
from neo4j import GraphDatabase, RoutingControl, basic_auth

In [12]:
import pandas as pd
import numpy as np

## Setup the Connection to the Database
Choose a database server, a database, credentials for accessing it, and use them to configure the driver.\
The driver helps connecting the database and passing the Cypher statements

In [13]:
# DB URI and authentication
URI = "bolt://localhost:7687"
AUTH = ("neo4j","Ola3Ola3")

In [14]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [15]:
driver.verify_connectivity()

### Query Templates

Execute a Cypher statement by creating a session and using the methods 
- Session.execute_read() and 
- Session.execute_write()

Do not hardcode or concatenate parameters: 
- use placeholders and 
- specify the parameters as keyword arguments

In [16]:
# template write request
def write_query(query, params={}):
    with driver.session(database="neo4j") as session:
        results = session.execute_write(lambda tx: tx.run(query, params).data())
        # response = [record.values()[0] for record in results]
        return results

In [17]:
# template read request
def read_query(query, params={}):
    with driver.session(database="neo4j") as session:
        results = session.execute_read(lambda tx: tx.run(query, params).data())       
        response = [record.values()[0] for record in results]
        return response

### File Reading Utilities

In [18]:
# Run countries before this

community_Wide_2017 = pd.read_csv('./Data/2017_-_Cities_Community_Wide_Emissions.csv')
emissions_reduction_2016 = pd.read_csv('./Data/2016_-_Cities_Emissions_Reduction_Targets_20240207.csv')
emissions_reduction_2017 = pd.read_csv("./Data/2017_-_Cities_Emissions_Reduction_Targets_20240207.csv")
citywide_ghg_2016 = pd.read_csv('./Data/2016_-_Citywide_GHG_Emissions_20240207.csv')
climate_risk = pd.read_csv('./Data/2023_Cities_Climate_Risk_and_Vulnerability_Assessments_20240207.csv')

In [19]:

def fillna_with_mode(series):
    # Calculate the mode while ignoring NaN values
    mode_series = series.dropna().mode()
    if not mode_series.empty:
        # If there's at least one mode, use the first one
        mode_value = mode_series.iloc[0]
    else:
        # Fallback strategy if there's no mode
        mode_value = "Some default value"  # Adjust this as necessary
    return series.fillna(mode_value)


In [20]:
country = emissions_reduction_2017[["Country","Country Location","Region"]]
country = pd.concat([country, emissions_reduction_2016[["Country","Country Location"]]])
country = pd.concat([country, community_Wide_2017[["Country","Country Location","Region"]]])
country = pd.concat([country, citywide_ghg_2016[["Country","Country Location"]]])

climate_risk = climate_risk.rename(columns={"Country/Area":"Country", "CDP Region": "Region"})
country = pd.concat([country, climate_risk[["Country","Region"]]])
country = country.rename(columns = {"Country":"name", "Country Location":"location", "Region":"region"})

country['location'] = country.groupby('name')['location'].transform(fillna_with_mode)
country['region'] = country.groupby('name')['region'].transform(fillna_with_mode)

In [21]:
country = country.drop_duplicates(subset= ["name"], keep='first').head(1000)
country = country.dropna(subset=["name"])


In [22]:
city = emissions_reduction_2016[["Organisation", "Country", "Account No", "C40", "City Location", "City Short Name"]]
city =city.rename(columns ={"Organisation":"city_name", "Country": "country_id", "Account No":"account_no", "City Short Name": "city_short_name", "City Location": "city_location", "C40":"c40"})
df2 = emissions_reduction_2017[["Organisation", "Country", "Account No", "C40", "City Location"]]
df2 =df2.rename(columns ={"Organisation":"city_name", "Country": "country_id", "Account No":"account_no",  "City Location": "city_location", "C40":"c40"})

city = pd.concat([city,df2])

city =city.drop_duplicates("account_no")
city["c40"]=city["c40"]=="C40"



In [23]:

for index, row in community_Wide_2017.iterrows():
    if row["Account number"] in city["account_no"].values:
        city.loc[city["account_no"] == row["Account number"], "land_area"] = row["​Land area (in square km)"]
        city.loc[city["account_no"] == row["Account number"], "average_altitude"] = row["​Average altitude (m)"]

    else:
        _row = pd.DataFrame({"city_name": [row["Organization"]],"country_id": [row["Country"]], "account_no": [row["Account number"]],
                             "city_short_name": [row["City"]], "c40": [row["C40"]]=="C40", "city_location": row[["City Location"]], "average_altitude": row["​Average altitude (m)"], "land_area": row["​Land area (in square km)"]})
        city = pd.concat([city, _row], ignore_index = True)

In [24]:
for index, row in citywide_ghg_2016.iterrows():
    if row["Account Number"] in city["account_no"].values:
        city.loc[city["account_no"] == row["Account Number"], "land_area"] = row["​Land area (in square km)"]
        city.loc[city["account_no"] == row["Account Number"], "average_altitude"] = row["Average altitude (m)"]

    else:
        _row = pd.DataFrame({"city_name": [row["City Name"]],"country_id": [row["Country"]], "account_no": [row["Account Number"]],
                             "city_short_name": [row["City Short Name"]], "c40": [row["C40"]]=="C40", "city_location": row[["City Location"]], "average_altitude": row["Average altitude (m)"], "land_area": row["​Land area (in square km)"]})
        city = pd.concat([city, _row], ignore_index = True)

In [25]:
city["gcom"] = None
for index, row in climate_risk.iterrows():
    if row["Organization Number"] in city["account_no"].values:
        city.loc[city["account_no"] == row["Organization Number"], "gcom"] = row["GCoM City"]
    else:
        _row = pd.DataFrame({"city_name": [row["Organization Name"]],"country_id": [row["Country"]], "account_no": [row["Organization Number"]],
                             "city_short_name": [row["City"]], "c40": [row["C40 City"]], "city_location": row[["City Location"]], "gcom": [row["GCoM City"]]})
        city = pd.concat([city, _row], ignore_index = True)


### Ingest the Data into the Database

#### Characters

In [31]:
create_db = """
MERGE (c:Country{name:$name})
    SET c.location = CASE trim(toString(c.location)) WHEN 'None' THEN null ELSE $location END
    SET c.region = CASE trim(toString(c.region)) WHEN 'None' THEN null ELSE $region END
"""

In [32]:
# Execute transaction
for index, row in country.iterrows():
    response = write_query(create_db, params = {'name':row['name'], 'location':row['location'], 'region':row['region']})

In [33]:
add_cities = '''
    MERGE (c:Country {name:$name})
    WITH c
    MERGE (g:City {account_no: $account_no, country: c.name}) 
        SET g.city_name = CASE trim(toString(g.city_name)) WHEN 'None' THEN null ELSE $city_name END
        SET g.city_short_name = CASE trim(toString(g.city_short_name)) WHEN 'None' THEN null ELSE $city_short_name END
        SET g.c40 = CASE trim(toString(g.c40)) WHEN 'None' THEN null ELSE $c40 END
        SET g.city_location = CASE trim(toString(g.city_location)) WHEN 'None' THEN null ELSE $city_location END
        SET g.gcom = CASE trim(toString(g.gcom)) WHEN 'None' THEN null ELSE $gcom END
    MERGE (g)-[:IS_IN]->(c)
    RETURN c
'''

In [35]:
city = city.dropna(subset="country_id")

In [36]:

# Execute transaction
for index, row in city.iterrows():
    response = write_query(add_cities, params = {'city_name':row['city_name'], 
                                                 'city_location':row['city_location'], 
                                                 'account_no':row['account_no'], 
                                                 'name':row['country_id'], 
                                                 'city_short_name':row['city_short_name'], 
                                                 'c40':row['c40'], 
                                                 'gcom':row['gcom']})

In [44]:
citywide_ghg_2016 = citywide_ghg_2016.rename(columns ={ "Account Number": "account_no", "Reporting Year": "reporting_year","Measurement Year":"measurement_year", "Boundary": "boundary", "Primary Methodology": "primary_methodology", "Methodology Details": "methodology_details", "Gases included": "gases", "Total City-wide Emissions (metric tonnes CO2e)": "total_emissions", "Total Scope 1 Emissions (metric tonnes CO2e)": "total_scope_1", "Total Scope 2 Emissions (metric tonnes CO2e)": "total_scope_2", "Increase/Decrease from last year": "increase_decrease", "Reason for increase/decrease in emissions": "reason_for_increase"})

In [47]:
add_ghg = '''
    MERGE (g:ghg {city_id:$account_no})
        SET g.reporting_year = CASE trim(toString(g.reporting_year)) WHEN 'None' THEN null ELSE $reporting_year END
        SET g.measurement_year = CASE trim(toString(g.measurement_year)) WHEN 'None' THEN null ELSE $measurement_year END
        SET g.boundary = CASE trim(toString(g.boundary)) WHEN 'None' THEN null ELSE $boundary END
        SET g.primary_methodology = CASE trim(toString(g.primary_methodology)) WHEN 'None' THEN null ELSE $primary_methodology END
        SET g.methodology_details = CASE trim(toString(g.methodology_details)) WHEN 'None' THEN null ELSE $methodology_details END
        SET g.gases = CASE trim(toString(g.gases)) WHEN 'None' THEN null ELSE $gases END
        SET g.total_emissions = CASE trim(toString(g.total_emissions)) WHEN 'None' THEN null ELSE $total_emissions END
        SET g.total_scope_1 = CASE trim(toString(g.total_scope_1)) WHEN 'None' THEN null ELSE $total_scope_1 END
        SET g.total_scope_2 = CASE trim(toString(g.total_scope_2)) WHEN 'None' THEN null ELSE $total_scope_2 END
        SET g.increase_decrease = CASE trim(toString(g.increase_decrease)) WHEN 'None' THEN null ELSE $increase_decrease END
        SET g.reason_for_increase = CASE trim(toString(g.reason_for_increase)) WHEN 'None' THEN null ELSE $reason_for_increase END
    WITH g
    MERGE (c:City {account_no: $account_no}) 
    MERGE (g)-[:FROM]->(c)
    RETURN g
'''

In [49]:

# Execute transaction
for index, row in citywide_ghg_2016.iterrows():
    response = write_query(add_ghg, params = { "account_no": row["account_no"],
            "reporting_year": row["reporting_year"],
            "measurement_year":row["measurement_year"],
            "boundary": row["boundary"],
            "primary_methodology":row["primary_methodology"],
            "methodology_details":row["methodology_details"],
            "gases": row["gases"].split(";") if pd.notna(row["gases"]) else [],
            "total_emissions": row["total_emissions"],
            "total_scope_1": row["total_scope_1"],
            "total_scope_2": row["total_scope_2"],
            "increase_decrease": row["increase_decrease"],
            "reason_for_increase":row["reason_for_increase"]
            })

In [51]:
climate_risk = climate_risk.rename(columns ={ "Organization Number": "city_id", "Primary author(s) of assessment": "author", "Confirm attachment/link provided": "confirmation","Assessment attachment and/or direct link":"attachment", "Access": "is_public", "Questionnaire": "questionnaire"})
climate_risk["is_public"] = climate_risk["is_public"] == "public"

In [52]:
add_climate_risk = '''
    MERGE (r:risk {city_id:$city_id})
        SET r.author = CASE trim(toString(r.author)) WHEN 'None' THEN null ELSE $author END
        SET r.confirmation = CASE trim(toString(r.confirmation)) WHEN 'None' THEN null ELSE $confirmation END
        SET r.attachment = CASE trim(toString(r.attachment)) WHEN 'None' THEN null ELSE $attachment END
        SET r.is_public = CASE trim(toString(r.is_public)) WHEN 'None' THEN null ELSE $is_public END
        SET r.questionnaire = CASE trim(toString(r.questionnaire)) WHEN 'None' THEN null ELSE $questionnaire END
    WITH r
    MERGE (c:City {account_no: $city_id})
    MERGE (r)-[:FROM]->(c)
    RETURN r
'''

In [53]:

# Execute transaction
for index, row in climate_risk.iterrows():
    response = write_query(add_climate_risk, params = { "city_id": row["city_id"],
            "author": row["author"],
            "confirmation":row["confirmation"],
            "attachment": row["attachment"],
            "is_public":row["is_public"],
            "questionnaire":row["questionnaire"],
            })

#### Relations

In [ ]:
file_name = 'HP-relations-seen.csv'
path = data_dir+file_name

In [ ]:
# with pandas
data = pd.read_csv(path, ',')
data

Unexpected exception formatting exception. Falling back to standard exception


C:\Users\Bruger\AppData\Local\Temp\ipykernel_18736\301918582.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  data = pd.read_csv(path, ',')
Traceback (most recent call last):
  File "C:\Users\Bruger\AppData\Roaming\Python\Python311\site-packages\IPython\core\formatters.py", line 221, in catch_format_error
    r = method(self, *args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bruger\AppData\Roaming\Python\Python311\site-packages\IPython\core\formatters.py", line 342, in __call__
    return method()
           ^^^^^^^^
  File "c:\Users\Bruger\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\frame.py", line 1106, in _repr_html_
  File "c:\Users\Bruger\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\io\formats\format.py", line 1110, in to_html
    When formatting an Index subclass
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users

               source            target   value
0     Petunia Dursley    Vernon Dursley   67629
1      Dudley Dursley    Vernon Dursley   30647
2      Dudley Dursley   Petunia Dursley  104637
3        Harry Potter    Vernon Dursley   30959
4        Harry Potter   Petunia Dursley  104646
..                ...               ...     ...
401    George Weasley     Poppy Pomfrey  100216
402    Nicolas Flamel  Perenelle Flamel  100608
403  Albus Dumbledore  Helena Ravenclaw  103872
404    Ronald Weasley    Vernon Dursley  104671
405  Hermione Granger    Vernon Dursley  104732

[406 rows x 3 columns]

In [ ]:
data.isnull().sum()

source    0
target    0
value     0
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  406 non-null    object
 1   target  406 non-null    object
 2   value   406 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.6+ KB


In [ ]:
datay = data.replace({np.nan: None})

In [ ]:
datay.sample(5)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Bruger\AppData\Roaming\Python\Python311\site-packages\IPython\core\formatters.py", line 221, in catch_format_error
    r = method(self, *args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bruger\AppData\Roaming\Python\Python311\site-packages\IPython\core\formatters.py", line 342, in __call__
    return method()
           ^^^^^^^^
  File "c:\Users\Bruger\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\frame.py", line 1106, in _repr_html_
  File "c:\Users\Bruger\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\io\formats\format.py", line 1110, in to_html
    When formatting an Index subclass
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Bruger\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
            ^^^^^^^^^^^^^
  File "c:\Users\Bruger\AppData\Local\Programs\Python\Python311\Lib\s

                 source                target  value
196        Harry Potter                Peeves  92920
132       Gregory Goyle          Harry Potter  75359
118    Alberic Grunnion  Hengist of Woodcroft  35543
145  Neville Longbottom                Trevor  92348
198  Neville Longbottom                Peeves  44475

In [ ]:
create_rel = """
    MERGE (s:Character{name:$source})
    MERGE (t:Character{name:$target})
    MERGE (s)-[r:INTERACTS]-(t)
    SET r.weight = $value
"""    

In [ ]:
# Execute transaction
for index, row in datay.iterrows():
    response = write_query(create_rel, 
                           params = {'source':row['source'], 'target':row['target'], 'value':row['value']})

## Query the Database

To enable permissions for running the algorithms on the localmachine, aadd this line to your config file:
__dbms.security.procedures.unrestricted=jwt.security.*, apoc.*, gds.*__

In [ ]:
def read_gds(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

### Community Detection

In [ ]:
# create a projection
query = """
    CALL gds.graph.project('dor', 'Character', {INTERACTS:{orientation:'UNDIRECTED'}})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS title, score AS rank
    ORDER BY rank DESCENDING, title LIMIT 5
"""

In [ ]:
response = read_gds(query)
response

In [ ]:
query = """
    CALL gds.louvain.stream('dom')
    YIELD nodeId, communityId, intermediateCommunityIds
    RETURN gds.util.asNode(nodeId).name AS name, communityId
    ORDER BY communityId ASC LIMIT 15
"""

In [ ]:
# writes the calculated community coeficients as parameters of the nodes
response = read_gds(query)
response

### Degree Centrality

In [ ]:
query = """
    CALL gds.degree.stream('dor')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS title, score AS connections
    ORDER BY rank DESCENDING, title LIMIT 5
"""

In [ ]:
response = read_gds(query)
response

### Page Rank

In [ ]:
# create projection
query = '''
CALL gds.graph.project(
  'dom',
  'Character',
  'INTERACTS',
  {
    relationshipProperties: 'weight'
  }
)
'''

In [ ]:
response = read_gds(query)
response

In [ ]:
query = """
    CALL gds.pageRank.stream('dom')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).title AS title, score
    ORDER BY score DESC
"""

In [ ]:
response = read_gds(query)
response

### Shortest Path

In [ ]:
# create projection
query = '''
CALL gds.graph.project(
    'pat',
    'Character',
    'INTERACTS',
    {
        relationshipProperties: 'weight'
    }
)
'''

In [ ]:
response = read_gds(query)
response

In [ ]:
query = """
    MATCH (source:Character {title: 'Harry Potter'}), (target:Character {title: 'Albus Dumbledore'})
    CALL gds.shortestPath.dijkstra.stream('pat', 
    {
        sourceNode: source,
        targetNode: target,
        relationshipWeightProperty: 'weight'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        index,
        gds.util.asNode(sourceNode).title AS sourceNodeName,
        gds.util.asNode(targetNode).title AS targetNodeName,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).title] AS nodeNames,
        costs,
        nodes(path) as path
    ORDER BY index
"""

In [ ]:
response = read_gds(query)
response